In [1]:
import torch
from torchinfo import summary

from torch.optim import Adam
from torch.nn import CrossEntropyLoss

from torchvision import transforms
from predict import predict
from train import train, test

In [3]:
PATCH_SIZE = 8 # P
IMAGE_CROP = 240
IMAGE_WIDTH = 224
IMAGE_CHANNELS = 1
BATCH_SIZE = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device, f"({torch.cuda.get_device_name(device)})" if torch.cuda.is_available() else "")

Using device: cuda (NVIDIA GeForce RTX 3060 Laptop GPU)


In [7]:
train_transform = transforms.Compose([
    # transforms.CenterCrop(IMAGE_CROP),
    transforms.Resize((IMAGE_WIDTH, IMAGE_WIDTH)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.1155], std=[0.2224])
])

test_transform = transforms.Compose([
    # transforms.CenterCrop(IMAGE_CROP),
    transforms.Resize((IMAGE_WIDTH, IMAGE_WIDTH)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.1167], std=[0.2228])
])

from dataset import create_dataloaders
root_dir = "C:/Users/Jacqu/Downloads/AD_NC"
train_loader, valid_loader, test_loader = create_dataloaders(root_dir, train_transform, test_transform, batch_size=BATCH_SIZE, datasplit=0.8)
print(f"Num iamges for train: {len(train_loader.dataset)}")
print(f"Num images for valid: {len(valid_loader.dataset)}")
print(f"Num images for test: {len(test_loader.dataset)}")

Num iamges for train: 17200
Num images for valid: 4320
Num images for test: 9000
